<table style="width:100%; background-color: #D9EDF7">
  <tr>
    <td style="border: 1px solid #CFCFCF">
      <b>Weather data: Main notebook</b>
      <ul>
        <li><a href="main.ipynb">Main Notebook</a></li>
        <li>Downloading Notebook</li>
      </ul>
      <br>This Notebook is part of the <a href="http://data.open-power-system-data.org/weather_data">Weather data Datapackage</a> of <a href="http://open-power-system-data.org">Open Power System Data</a>.
    </td>
  </tr>
</table>

# Table of Contents
 <p><div class="lev1"><a href="#How-to-obtain-weather-data-from-MERRA-2-(Part-2):-Download-raw-data"><span class="toc-item-num">1&nbsp;&nbsp;</span>How to obtain weather data from MERRA-2 (Part 2): Download raw data</a></div><div class="lev2"><a href="#About-this-Notebook"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>About this Notebook</a></div><div class="lev3"><a href="#Other-notebooks"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Other notebooks</a></div><div class="lev3"><a href="#License"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>License</a></div><div class="lev3"><a href="#Table-of-contents"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Table of contents</a></div><div class="lev1"><a href="#Script-Setup"><span class="toc-item-num">2&nbsp;&nbsp;</span>Script Setup</a></div><div class="lev1"><a href="#Download-raw-data"><span class="toc-item-num">3&nbsp;&nbsp;</span>Download raw data</a></div><div class="lev2"><a href="#Input"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Input</a></div><div class="lev3"><a href="#Parameters-choices"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Parameters choices</a></div><div class="lev3"><a href="#Timeframe"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Timeframe</a></div><div class="lev3"><a href="#Geography-coordinates"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Geography coordinates</a></div><div class="lev2"><a href="#Subsetting-data"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Subsetting data</a></div><div class="lev2"><a href="#Downloading-data"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Downloading data</a></div><div class="lev1"><a href="#Setting-up-dataframe(s)"><span class="toc-item-num">4&nbsp;&nbsp;</span>Setting up dataframe(s)</a></div><div class="lev2"><a href="#Concatenating/combining-individual-files"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Concatenating/combining individual files</a></div><div class="lev2"><a href="#First-look-at-the-final-data-frame-structure-and-format"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>First look at the final data frame structure and format</a></div><div class="lev2"><a href="#Saving-dataframe"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Saving dataframe</a></div>

# Introductory Notes
This script contains code that allows the download, subset and processing of [MERRA-2](http://gmao.gsfc.nasa.gov/reanalysis/MERRA-2/) datasets (provided by NASA Goddard Space Flight Center) and export them as CSV.

**Weather data differ significantly from the other data types used resp. provided by OPSD** in that the sheer size of the data packages greatly exceeds OPSD's capacity to host them in a similar way as feed-in timeseries, power plant data etc. While the other data packages also offer a complete one-klick download of the bundled data packages with all relevant data this is impossible for weather datasets like MERRA-2 due to their size (variety of variables, very long timespan, huge geographical coverage etc.). It would make no sense to mirror the data from the NASA servers.

Instead we choose to provide only a **documented methodological script**. The  method describes one way to automatically obtain the desired weather data from the MERRA-2 database and simplifies resp. unifies alternative manual data obtaining methods in a single script.

**More detailed background information** on weather data can be found in the <a href="main.ipynb">Main notebook</a> and the [OPSD Wiki](https://github.com/Open-Power-System-Data/common/wiki/Information-on-weather-data) on Github.

## State of the script:
The script is still in development. It is currently working. The next step will be adding support for more datasets and more weather variables. 

## How to use the script:
To download MERRA-2 data, you have to register at NASA earth data portal
1. Register an account at [https://urs.earthdata.nasa.gov/](https://urs.earthdata.nasa.gov/)
2. Go to my applications and add NASA GESDISC DATA ARCHIVE
3. Input your username and password when requested by the script

---
# Script Setup

In [22]:
import pandas as pd
import xarray as xr
import numpy as np
import requests
import logging
import os
from datetime import datetime
from calendar import monthrange
from opendap_download.multi_processing_download import DownloadManager
import math
from functools import partial
import re
import getpass
from datetime import datetime, timedelta
import pytz
import dateutil.parser

# Set up a log
logging.basicConfig(level=logging.INFO)
log = logging.getLogger('notebook')


---
# Download raw data

## Input
This part defines the input parameters according to the user and creates an URL that can download the desired MERRA-2 data via the OPeNDAP interface ([more information)](https://github.com/Open-Power-System-Data/common/wiki/Information-on-weather-data#4-what-is-opendap).

### Parameter selection 
These are the possible parameters (i.e. weather data)
* wind
* solar radiation
* temperature

If you want to select more than one parameter, separate them with commas. For example: `wind, solar radiation, temperature`

In [23]:
# Getting user input
# This version only supports wind so far. This line does not do anything at 
# this point.
possible_params = ['wind', 'solar radiation', 'temperature']

### Timeframe

Definition of desired timespan the data is needed for. (currently only full years possible)

In [24]:
# User input of timespan
download_year = 2014
# Create the start date 2014-01-01
download_start_date = str(download_year) + '-01-01'

### Geography coordinates
Definition of desired coordinates. The user has to input two corner coordinates of a rectangular area (Format WGS84, decimal system).
* Northeast coordinate: lat_1, lon_1
* Southwest coordinate: lat_2, lon_2

The area/coordinates will be converted from lat/lon to the MERRA-2 grid coordinates.
Since the resolution of the MERRA-2 grid is 0.5 x 0.625°, the given coordinates will 
matched as close as possible.

In [25]:
# User input of coordinates
# ------
# Example: Schleswig-Holstein (lat/lon)
# Northeastern point: 55,036823°N, 11,349297°E
# Southwestern point: 53,366266°N, 7,887088°E

# Northeastern coordinate 
lat_1, lon_1 = 53.366266, 7.887088
# Southwestern coordinate
lat_2, lon_2 = 55.036823, 11.349297

def translate_lat_to_geos5_native(latitude):
    """
    The source for this formula is in the MERRA2 
    Variable Details - File specifications for GEOS pdf file.
    
    latitude: float Needs +/- instead of N/S
    """
    return 1 + ((latitude + 90) / 0.5)

def translate_lon_to_geos5_native(longitude):
    """See function above"""
    return (1 + ((longitude) + 180) / 0.625)

def find_closest_coordinate(calc_coord, coord_array):
    """
    Since the resolution of the grid is 0.5 x 0.625, the 'real world'
    coordinates will not be matched 100% correctly. This function matches 
    the coordinates as close as possible. 
    """
    # np.argmin() finds the smallest value in an array and returns its
    # index. np.abs() returns the absolute value of each item of an array.
    # To summarize, the function finds the difference closest to 0 and returns 
    # its index. 
    index = np.abs(coord_array-calc_coord).argmin()
    return coord_array[index]

# The arrays contain the coordinates of the grid used by the API.
lat_coords = np.arange(0, 360, dtype=int)
lon_coords = np.arange(0, 575, dtype=int)

# Translate the coordinates that define your area to grid coordinates.
lat_coord_1 = translate_lat_to_geos5_native(lat_1)
lon_coord_1 = translate_lon_to_geos5_native(lon_1)
lat_coord_2 = translate_lat_to_geos5_native(lat_2)
lon_coord_2 = translate_lon_to_geos5_native(lon_2)


# Find the closest coordinate in the grid.
lat_co_1_closest = find_closest_coordinate(lat_coord_1, lat_coords)
lon_co_1_closest = find_closest_coordinate(lon_coord_1, lon_coords)
lat_co_2_closest = find_closest_coordinate(lat_coord_2, lat_coords)
lon_co_2_closest = find_closest_coordinate(lon_coord_2, lon_coords)

# Check the precision of the grid coordinates. These coordinates are not lat/lon. 
# They are coordinates on the MERRA-2 grid. 
log.info('Calculated coordinates for point 1: ' + str((lat_coord_1, lon_coord_1)))
log.info('Closest coordinates for point 1: ' + str((lat_co_1_closest, lon_co_1_closest)))
log.info('Calculated coordinates for point 2: ' + str((lat_coord_2, lon_coord_2)))
log.info('Closest coordinates for point 2: ' + str((lat_co_2_closest, lon_co_2_closest)))




INFO:notebook:Calculated coordinates for point 1: (287.732532, 301.61934080000003)
INFO:notebook:Closest coordinates for point 1: (288, 302)
INFO:notebook:Calculated coordinates for point 2: (291.073646, 307.1588752)
INFO:notebook:Closest coordinates for point 2: (291, 307)


## Subsetting data

Combining parameter choices above/translation according to OPenDAP guidelines into URL-appendix

In [26]:
def translate_year_to_file_number(year):
    """
    The file names consist of a number and a meta data string. 
    The number changes over the years. 1980 until 1991 it is 100, 
    1992 until 2000 it is 200, 2001 until 2010 it is  300 
    and from 2011 until now it is 400.
    """
    file_number = ''
    
    if year >= 1980 and year < 1992:
        file_number = '100'
    elif year >= 1992 and year < 2001:
        file_number = '200'
    elif year >= 2001 and year < 2011:
        file_number = '300'
    elif year >= 2011:
        file_number = '400'
    else:
        raise Exception('The specified year is out of range.')
    
    return file_number
    


def generate_url_params(parameter, time_para, lat_para, lon_para):
    """Creates a string containing all the parameters in query form"""
    parameter = map(lambda x: x + time_para, parameter)
    parameter = map(lambda x: x + lat_para, parameter)
    parameter = map(lambda x: x + lon_para, parameter)
    
    return ','.join(parameter)
    
    

def generate_download_links(download_years, base_url, dataset_name, url_params):
    """
    Generates the links for the download. 
    download_years: The years you want to download as array. 
    dataset_name: The name of the data set. For example tavg1_2d_slv_Nx
    """
    urls = []
    for y in download_years: 
    # build the file_number
        y_str = str(y)
        file_num = translate_year_to_file_number(download_year)
        for m in range(1,13):
            # build the month string: for the month 1 - 9 it starts with a leading 0. 
            # zfill solves that problem
            m_str = str(m).zfill(2)
            # monthrange returns the first weekday and the number of days in a 
            # month. Also works for leap years.
            _, nr_of_days = monthrange(y, m)
            for d in range(1,nr_of_days+1):
                d_str = str(d).zfill(2)
                # Create the file name string
                file_name = 'MERRA2_{num}.{name}.{y}{m}{d}.nc4'.format(
                    num=file_num, name=dataset_name, 
                    y=y_str, m=m_str, d=d_str)
                # Create the query
                query = '{base}{y}/{m}/{name}.nc4?{params}'.format(
                    base=base_url, y=y_str, m=m_str, 
                    name=file_name, params=url_params)
                urls.append(query)
    return urls

requested_params = ['U2M', 'U10M', 'U50M', 'V2M', 'V10M', 'V50M', 'DISPH']
requested_time = '[0:1:23]'
# Creates a string that looks like [start:1:end]. start and end are the lat or
# lon coordinates define your area.
requested_lat = '[{lat_1}:1:{lat_2}]'.format(
                lat_1=lat_co_1_closest, lat_2=lat_co_2_closest)
requested_lon = '[{lon_1}:1:{lon_2}]'.format(
                lon_1=lon_co_1_closest, lon_2=lon_co_2_closest)



parameter = generate_url_params(requested_params, requested_time,
                                requested_lat, requested_lon)

BASE_URL = 'http://goldsmr4.sci.gsfc.nasa.gov:80/opendap/MERRA2/M2T1NXSLV.5.12.4/'
generated_URL = generate_download_links([download_year], BASE_URL, 
                                        'tavg1_2d_slv_Nx', 
                                        parameter)
            
# See what a query to the MERRA-2 portal looks like.        
log.info(generated_URL[0])

INFO:notebook:http://goldsmr4.sci.gsfc.nasa.gov:80/opendap/MERRA2/M2T1NXSLV.5.12.4/2014/01/MERRA2_400.tavg1_2d_slv_Nx.20140101.nc4.nc4?U2M[0:1:23][288:1:291][302:1:307],U10M[0:1:23][288:1:291][302:1:307],U50M[0:1:23][288:1:291][302:1:307],V2M[0:1:23][288:1:291][302:1:307],V10M[0:1:23][288:1:291][302:1:307],V50M[0:1:23][288:1:291][302:1:307],DISPH[0:1:23][288:1:291][302:1:307]


---
# Downloading data
This part subsequently downloads the subsetted raw data from the MERRA-2-datasets. 
The download process is outsourced from the notebook, because it is a standard and repetitive process. If you are interested in the the code, see the [opendap_download module](opendap_download/)

## Get wind data
from the dataset [tavg1_2d_slv_Nx (M2T1NXSLV)](http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/contents.html)

In [27]:
# Download data (one file per day and dataset) with links to local directory.
# Username and password for MERRA-2 (NASA earthdata portal)
username = input('Username: ')
password = getpass.getpass('Password:')
# The DownloadManager is able to download files. If you have a fast internet 
# connection, setting this to 2 is enough. If you have slow wifi, try setting
# it to 4 or 5. If you download too fast, the data portal might ban you for a 
# day. 
NUMBER_OF_CONNECTIONS = 2

# The DownloadManager class is defined in the opendap_download module. 
download_manager = DownloadManager()
download_manager.set_username_and_password(username, password)
download_manager.download_path = 'download'
download_manager.download_urls = generated_URL
# If you want to see the download progress, check the download folder you 
# specified
%time download_manager.start_download(NUMBER_OF_CONNECTIONS)

Username: janurb
Password:········
Wall time: 17min 7s


## Get roughness
from the dataset [tavg1_2d_flx_Nx (M2T1NXFLX)](http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXFLX.5.12.4/contents.html) 

In [28]:
# Roughness data is in a different data set. The parameter is called Z0M. 
roughness_para = generate_url_params(['Z0M'], requested_time, 
                                     requested_lat, requested_lon)
ROUGHNESS_BASE_URL = 'http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXFLX.5.12.4/'
roughness_links = generate_download_links([download_year], ROUGHNESS_BASE_URL,
                                          'tavg1_2d_flx_Nx', roughness_para)
            
download_manager.download_path = 'roughness_download'
download_manager.download_urls = roughness_links

# If you want to see the download progress, check the download folder you 
# specified.
%time download_manager.start_download(NUMBER_OF_CONNECTIONS)

Wall time: 13min 12s


## Get lat and lon dimensions
For now, the dataset only has MERRA-2 grid coordinates. To translate the points
back to "real world" coordinates, the data portal offers a dimension scale file.

In [29]:
# The dimensions map the MERRA2 grid coordinates to lat/lon. The coordinates 
# to request are 0:360 wheare as the other coordinates are 1:361
requested_lat_dim = '[{lat_1}:1:{lat_2}]'.format(
                    lat_1=lat_co_1_closest - 1, lat_2=lat_co_2_closest - 1)
requested_lon_dim = '[{lon_1}:1:{lon_2}]'.format(
                    lon_1=lon_co_1_closest - 1, lon_2=lon_co_2_closest - 1)

lat_lon_dimension_para = 'lat' + requested_lat_dim + ',lon' + requested_lon_dim
# Creating the download url.
dimension_url = 'http://goldsmr4.sci.gsfc.nasa.gov:80/opendap/MERRA2/M2T1NXSLV.5.12.4/2014/01/MERRA2_400.tavg1_2d_slv_Nx.20140101.nc4.nc4?'
dimension_url = dimension_url + lat_lon_dimension_para
download_manager.download_path = 'dimension_scale'
download_manager.download_urls = [dimension_url]
# Since the dimension is only one file, we only need one connection. 
%time download_manager.start_download(1)


Wall time: 4.1 s


## Check the precision of the downloaded data
Due to the back and forth conversion from "real world" coordinates to MERRA-2 grid points,
this part helps you to check if the conversion was precise enough. 

In [30]:
file_path = os.path.join('dimension_scale', DownloadManager.get_filename(
        dimension_url))

with xr.open_dataset(file_path) as ds_dim:
    df_dim = ds_dim.to_dataframe()

lat_array = ds_dim['lat'].data.tolist()
lon_array = ds_dim['lon'].data.tolist()

# The log output helps evaluating the precision of the received data.
log.info('Requested lat: ' + str((lat_1, lat_2)))
log.info('Received lat: ' + str(lat_array))
log.info('Requested lon: ' + str((lon_1, lon_2)))
log.info('Received lon: ' + str(lon_array))

INFO:notebook:Requested lat: (53.366266, 55.036823)
INFO:notebook:Received lat: [53.5, 54.0, 54.5, 55.0]
INFO:notebook:Requested lon: (7.887088, 11.349297)
INFO:notebook:Received lon: [8.125, 8.75, 9.375, 10.0, 10.625, 11.25]


---
# Setting up DataFrame
This part sets up a DataFrame and reads the raw data into it.

In [31]:
def extract_date(data_set):
    """
    Extracts the date from the filename before merging the datasets. 
    """
    try:
        # The attribute name changed during the development of this script
        # from HDF5_Global.Filename to Filename. 
        if 'HDF5_GLOBAL.Filename' in data_set.attrs:
            f_name = data_set.attrs['HDF5_GLOBAL.Filename']
        elif 'Filename' in data_set.attrs:
            f_name = data_set.attrs['Filename']
        else: 
            raise AttributeError('The attribute name has changed again!')
        
        # find a match between "." and ".nc4" that does not have "." .
        exp = r'(?<=\.)[^\.]*(?=\.nc4)'
        res = re.search(exp, f_name).group(0)
        # Extract the date. 
        y, m, d = res[0:4], res[4:6], res[6:8]
        date_str = ('%s-%s-%s' % (y, m, d))
        data_set = data_set.assign(date=date_str)
        return data_set

    except KeyError:
        # The last dataset is the one all the other sets will be merged into. 
        # Therefore, no date can be extracted.
        return data_set
        

file_path = os.path.join('download', '*.nc4')

try:
    with xr.open_mfdataset(file_path, concat_dim='date',
                           preprocess=extract_date) as ds_wind:
        print(ds_wind)
        df_wind = ds_wind.to_dataframe()
        
except xr.MergeError as e:
    print(e)

<xarray.Dataset>
Dimensions:  (date: 365, lat: 4, lon: 6, time: 24)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4 5
  * date     (date) <U10 '2014-01-01' '2014-01-02' '2014-01-03' '2014-01-04' ...
Data variables:
    DISPH    (date, time, lat, lon) float64 0.141 0.2787 0.2898 0.245 0.1877 ...
    U10M     (date, time, lat, lon) float64 -1.18 -1.277 -1.059 -1.574 ...
    V50M     (date, time, lat, lon) float64 9.038 8.616 8.948 8.863 8.753 ...
    U50M     (date, time, lat, lon) float64 -1.312 -1.587 -1.283 -1.846 ...
    V2M      (date, time, lat, lon) float64 5.821 4.311 4.231 4.776 5.176 ...
    V10M     (date, time, lat, lon) float64 7.409 6.227 6.227 6.653 6.888 ...
    U2M      (date, time, lat, lon) float64 -0.9013 -0.885 -0.7202 -1.137 ...


In [32]:
df_wind.reset_index(inplace=True)

In [33]:
start_date = datetime.strptime(download_start_date, '%Y-%m-%d')

def calculate_datetime(d_frame):
    """
    Calculates the accumulated hour based on the date.
    """
    cur_date = datetime.strptime(d_frame['date'], '%Y-%m-%d')
    hour = int(d_frame['time'])
    delta = abs(cur_date - start_date).days
    date_time_value = (delta * 24) + (hour)
    return date_time_value


df_wind['date_time_hours'] = df_wind.apply(calculate_datetime, axis=1)
df_wind

,date,lat,lon,time,DISPH,U10M,V50M,U50M,V2M,V10M,U2M,date_time_hours
0,2014-01-01,0,0,0,0.140991,-1.179733,9.038284,-1.311984,5.820765,7.409066,-0.901322,0
1,2014-01-01,0,0,1,0.140930,-0.879409,8.794005,-0.840221,5.633766,7.161377,-0.666895,1
2,2014-01-01,0,0,2,0.140930,-0.919946,8.179359,-0.993543,5.254070,6.650461,-0.697199,2
3,2014-01-01,0,0,3,0.140869,-0.624578,8.161971,-0.647694,5.308243,6.702684,-0.449956,3
4,2014-01-01,0,0,4,0.140869,0.091913,8.392127,0.262211,5.492879,6.932301,0.115220,4
5,2014-01-01,0,0,5,0.140808,0.717062,8.293762,1.017545,5.456675,6.883762,0.607727,5
6,2014-01-01,0,0,6,0.140808,1.042843,8.050136,1.427836,5.329176,6.724246,0.860338,6
7,2014-01-01,0,0,7,0.140747,1.256459,7.979958,1.650132,5.266450,6.661009,1.029762,7
8,2014-01-01,0,0,8,0.140686,1.718015,8.161178,2.281217,5.338470,6.780849,1.383076,8
9,2014-01-01,0,0,9,0.140686,2.468855,8.092620,3.251890,5.253557,6.701610,1.947835,9


## Converting the timeformat to ISO 8601

In [34]:
def converting_timeformat_to_ISO8601(row):
    """Generates datetime according to ISO 8601 (UTC)"""
    date = dateutil.parser.parse(row['date'])
    hour = int(row['time'])
    # timedelta from the datetime module enables the programmer 
    # to add time to a date. 
    date_time = date + timedelta(hours = hour)
    return str(date_time.isoformat()) + 'Z'  # MERRA2 datasets have UTC time zone.
df_wind['date_utc'] = df_wind.apply(converting_timeformat_to_ISO8601, axis=1)


df_wind['date_utc']

0         2014-01-01T00:00:00Z
1         2014-01-01T01:00:00Z
2         2014-01-01T02:00:00Z
3         2014-01-01T03:00:00Z
4         2014-01-01T04:00:00Z
5         2014-01-01T05:00:00Z
6         2014-01-01T06:00:00Z
7         2014-01-01T07:00:00Z
8         2014-01-01T08:00:00Z
9         2014-01-01T09:00:00Z
10        2014-01-01T10:00:00Z
11        2014-01-01T11:00:00Z
12        2014-01-01T12:00:00Z
13        2014-01-01T13:00:00Z
14        2014-01-01T14:00:00Z
15        2014-01-01T15:00:00Z
16        2014-01-01T16:00:00Z
17        2014-01-01T17:00:00Z
18        2014-01-01T18:00:00Z
19        2014-01-01T19:00:00Z
20        2014-01-01T20:00:00Z
21        2014-01-01T21:00:00Z
22        2014-01-01T22:00:00Z
23        2014-01-01T23:00:00Z
24        2014-01-01T00:00:00Z
25        2014-01-01T01:00:00Z
26        2014-01-01T02:00:00Z
27        2014-01-01T03:00:00Z
28        2014-01-01T04:00:00Z
29        2014-01-01T05:00:00Z
                  ...         
210210    2014-12-31T18:00:00Z
210211  

In [35]:
def calculate_windspeed(d_frame, idx_u, idx_v):
    """
    Calculates the windspeed. The returned unit is m/s
    """
    um = int(d_frame[idx_u])
    vm = int(d_frame[idx_v])
    speed = math.sqrt((um ** 2) + (vm ** 2))
    return round(speed, 2)

# partial is used to create a function with pre-set arguments. 
calc_windspeed_2m = partial(calculate_windspeed, idx_u='U2M', idx_v='V2M')
calc_windspeed_10m = partial(calculate_windspeed, idx_u='U10M', idx_v='V10M')
calc_windspeed_50m = partial(calculate_windspeed, idx_u='U50M', idx_v='V50M')

df_wind['v_2m'] = df_wind.apply(calc_windspeed_2m, axis=1)
df_wind['v_10m']= df_wind.apply(calc_windspeed_10m, axis=1)
df_wind['v_50m'] = df_wind.apply(calc_windspeed_50m, axis=1)
df_wind

,date,lat,lon,time,DISPH,U10M,V50M,U50M,V2M,V10M,U2M,date_time_hours,date_utc,v_2m,v_10m,v_50m
0,2014-01-01,0,0,0,0.140991,-1.179733,9.038284,-1.311984,5.820765,7.409066,-0.901322,0,2014-01-01T00:00:00Z,5.00,7.07,9.06
1,2014-01-01,0,0,1,0.140930,-0.879409,8.794005,-0.840221,5.633766,7.161377,-0.666895,1,2014-01-01T01:00:00Z,5.00,7.00,8.00
2,2014-01-01,0,0,2,0.140930,-0.919946,8.179359,-0.993543,5.254070,6.650461,-0.697199,2,2014-01-01T02:00:00Z,5.00,6.00,8.00
3,2014-01-01,0,0,3,0.140869,-0.624578,8.161971,-0.647694,5.308243,6.702684,-0.449956,3,2014-01-01T03:00:00Z,5.00,6.00,8.00
4,2014-01-01,0,0,4,0.140869,0.091913,8.392127,0.262211,5.492879,6.932301,0.115220,4,2014-01-01T04:00:00Z,5.00,6.00,8.00
5,2014-01-01,0,0,5,0.140808,0.717062,8.293762,1.017545,5.456675,6.883762,0.607727,5,2014-01-01T05:00:00Z,5.00,6.00,8.06
6,2014-01-01,0,0,6,0.140808,1.042843,8.050136,1.427836,5.329176,6.724246,0.860338,6,2014-01-01T06:00:00Z,5.00,6.08,8.06
7,2014-01-01,0,0,7,0.140747,1.256459,7.979958,1.650132,5.266450,6.661009,1.029762,7,2014-01-01T07:00:00Z,5.10,6.08,7.07
8,2014-01-01,0,0,8,0.140686,1.718015,8.161178,2.281217,5.338470,6.780849,1.383076,8,2014-01-01T08:00:00Z,5.10,6.08,8.25
9,2014-01-01,0,0,9,0.140686,2.468855,8.092620,3.251890,5.253557,6.701610,1.947835,9,2014-01-01T09:00:00Z,5.10,6.32,8.54


# Setting up Roughness dataframe

In [36]:
file_path = os.path.join('roughness_download', '*.nc4')
with xr.open_mfdataset(file_path, concat_dim='date', 
                       preprocess=extract_date) as ds_rough:
    df_rough = ds_rough.to_dataframe()

df_rough.reset_index(inplace=True)

df_rough

,date,lat,lon,time,Z0M
0,2014-01-01,0,0,0,0.027990
1,2014-01-01,0,0,1,0.027967
2,2014-01-01,0,0,2,0.027936
3,2014-01-01,0,0,3,0.027936
4,2014-01-01,0,0,4,0.027944
5,2014-01-01,0,0,5,0.027936
6,2014-01-01,0,0,6,0.027921
7,2014-01-01,0,0,7,0.027913
8,2014-01-01,0,0,8,0.027913
9,2014-01-01,0,0,9,0.027906


## Combining the wind and roughness dataframe

In [37]:
df = pd.merge(df_wind, df_rough, on=['date', 'lat', 'lon', 'time'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210240 entries, 0 to 210239
Data columns (total 17 columns):
date               210240 non-null object
lat                210240 non-null int64
lon                210240 non-null int64
time               210240 non-null int64
DISPH              210240 non-null float64
U10M               210240 non-null float64
V50M               210240 non-null float64
U50M               210240 non-null float64
V2M                210240 non-null float64
V10M               210240 non-null float64
U2M                210240 non-null float64
date_time_hours    210240 non-null int64
date_utc           210240 non-null object
v_2m               210240 non-null float64
v_10m              210240 non-null float64
v_50m              210240 non-null float64
Z0M                210240 non-null float64
dtypes: float64(11), int64(4), object(2)
memory usage: 28.9+ MB


---
# Structure the dataframe, add and remove columns

## Calculating the height with displacement height

In [38]:
# Calculate height for v_2m and v_10m (2 + DISPH or 10 + DISPH).
df['h_v1'] = df.apply((lambda x:int(x['DISPH']) + 2), axis=1)
df['h_v2'] = df.apply((lambda x:int(x['DISPH']) + 10), axis=1)

## Adding needed and removing not needed columns

In [39]:
df['v_100m'] = np.nan
df.drop('DISPH', axis=1, inplace=True)
df.drop(['time', 'date'], axis=1, inplace=True)
df.drop(['U2M', 'U10M', 'U50M', 'V2M', 'V10M', 'V50M'], axis=1, inplace=True)

df['lat'] = df['lat'].apply(lambda x: lat_array[int(x)])
df['lon'] = df['lon'].apply(lambda x: lon_array[int(x)])
df

,lat,lon,date_time_hours,date_utc,v_2m,v_10m,v_50m,Z0M,h_v1,h_v2,v_100m
0,53.5,8.125,0,2014-01-01T00:00:00Z,5.00,7.07,9.06,0.027990,2,10,NaN
1,53.5,8.125,1,2014-01-01T01:00:00Z,5.00,7.00,8.00,0.027967,2,10,NaN
2,53.5,8.125,2,2014-01-01T02:00:00Z,5.00,6.00,8.00,0.027936,2,10,NaN
3,53.5,8.125,3,2014-01-01T03:00:00Z,5.00,6.00,8.00,0.027936,2,10,NaN
4,53.5,8.125,4,2014-01-01T04:00:00Z,5.00,6.00,8.00,0.027944,2,10,NaN
5,53.5,8.125,5,2014-01-01T05:00:00Z,5.00,6.00,8.06,0.027936,2,10,NaN
6,53.5,8.125,6,2014-01-01T06:00:00Z,5.00,6.08,8.06,0.027921,2,10,NaN
7,53.5,8.125,7,2014-01-01T07:00:00Z,5.10,6.08,7.07,0.027913,2,10,NaN
8,53.5,8.125,8,2014-01-01T08:00:00Z,5.10,6.08,8.25,0.027913,2,10,NaN
9,53.5,8.125,9,2014-01-01T09:00:00Z,5.10,6.32,8.54,0.027906,2,10,NaN


## Renaming columns

In [40]:
rename_map = {'date_time_hours': 'cumulated hours', 
              'date_utc': 'timestamp',
              'v_2m': 'v1', 
              'v_10m': 'v2', 
              'Z0M': 'z0'
             }

df.rename(columns=rename_map, inplace=True)

# Change order of the columns
columns = ['timestamp', 'cumulated hours', 'lat', 'lon',
        'v1', 'v2', 'v_50m', 'v_100m',
        'h_v1', 'h_v2', 'z0']
df = df[columns]

---
## First look at the final data frame structure and format

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210240 entries, 0 to 210239
Data columns (total 11 columns):
timestamp          210240 non-null object
cumulated hours    210240 non-null int64
lat                210240 non-null float64
lon                210240 non-null float64
v1                 210240 non-null float64
v2                 210240 non-null float64
v_50m              210240 non-null float64
v_100m             0 non-null float64
h_v1               210240 non-null int64
h_v2               210240 non-null int64
z0                 210240 non-null float64
dtypes: float64(7), int64(3), object(1)
memory usage: 19.2+ MB


---
Take a look at the resulting dataframe

In [42]:
df

,timestamp,cumulated hours,lat,lon,v1,v2,v_50m,v_100m,h_v1,h_v2,z0
0,2014-01-01T00:00:00Z,0,53.5,8.125,5.00,7.07,9.06,NaN,2,10,0.027990
1,2014-01-01T01:00:00Z,1,53.5,8.125,5.00,7.00,8.00,NaN,2,10,0.027967
2,2014-01-01T02:00:00Z,2,53.5,8.125,5.00,6.00,8.00,NaN,2,10,0.027936
3,2014-01-01T03:00:00Z,3,53.5,8.125,5.00,6.00,8.00,NaN,2,10,0.027936
4,2014-01-01T04:00:00Z,4,53.5,8.125,5.00,6.00,8.00,NaN,2,10,0.027944
5,2014-01-01T05:00:00Z,5,53.5,8.125,5.00,6.00,8.06,NaN,2,10,0.027936
6,2014-01-01T06:00:00Z,6,53.5,8.125,5.00,6.08,8.06,NaN,2,10,0.027921
7,2014-01-01T07:00:00Z,7,53.5,8.125,5.10,6.08,7.07,NaN,2,10,0.027913
8,2014-01-01T08:00:00Z,8,53.5,8.125,5.10,6.08,8.25,NaN,2,10,0.027913
9,2014-01-01T09:00:00Z,9,53.5,8.125,5.10,6.32,8.54,NaN,2,10,0.027906


## Saving dataframe
Save the final dataframe locally

In [43]:
df.to_csv('weather_script_result.csv', index=False)